In [1]:
#import pandas as pd
import cudf
import cugraph
import cupy as cp

In [2]:
import graphistry
graphistry.__version__

'0+unknown'

In [3]:
graphistry.register(
    api=3,
    personal_key_id='',
    personal_key_secret=''
)

## Data

In [4]:
e_df = cudf.read_csv(
    'https://raw.githubusercontent.com/graphistry/pygraphistry/refs/heads/master/demos/data/facebook_combined.txt',
    sep=' ',
    names=['s', 'd']
)

e_df['s'] = e_df['s'].astype(str) + '_n'
e_df['d'] = e_df['d'].astype(str) + '_n'

R = 1000
#e_df = cudf.concat([
#    e_df,
#    cudf.DataFrame({
#        's': [f'rr{i}' for i in range(R)],
#        'd': [f'rr{i}' for i in range(R)]
#    })
#], ignore_index=True, sort=False).reset_index(drop=True)

print(e_df.shape)
e_df.head()

(88234, 2)


,s,d
0,0_n,1_n
1,0_n,2_n
2,0_n,3_n
3,0_n,4_n
4,0_n,5_n


In [5]:
g = graphistry.edges(e_df.assign(w=1.0), 's', 'd')#.compute_cugraph('ecg', directed=False)

In [6]:
%%time
G = cugraph.Graph(directed=True)
G.from_cudf_edgelist(g._edges, source=g._source, destination=g._destination, edge_attr='w')

CPU times: user 129 ms, sys: 56.2 ms, total: 185 ms
Wall time: 186 ms


In [7]:
g_ecg = g.compute_cugraph('ecg', directed=True, G=G)

In [8]:
g_ecg._nodes.head()

,id,ecg
0,3952_n,125
1,3953_n,169
2,3954_n,125
3,3955_n,12
4,3958_n,80


In [9]:
K = 10
top_k_ecg = g_ecg._nodes.ecg.value_counts()[:10].reset_index()
top_k_ecg

,index,ecg
0,182,490
1,66,486
2,46,379
3,19,282
4,48,236
5,87,231
6,62,227
7,124,205
8,169,186
9,86,182


In [10]:
g_ecg._nodes.sort_values(by='id').reset_index(drop=True).groupby(['ecg']).cumcount().reset_index(drop=True)

0         0
1         0
2         1
3         0
4         0
       ... 
4034    376
4035    377
4036    378
4037    229
4038    230
Length: 4039, dtype: int64

In [11]:
top_k_ecg.index % 3

Int64Index([0, 1, 2, 0, 1, 2, 0, 1, 2, 0], dtype='int64')

In [12]:
top_k_ecg.iloc[1,0], top_k_ecg.iloc[1,1]

(66, 486)

In [13]:

N = 100000
e2_df = cudf.DataFrame({
    's': [f'{i}_nn' for i in range(N)],
    'd': [f'{i}_nn' for i in range(N)],
    'w': 1.0
})
ecg_min = g_ecg._nodes.ecg.min()
ecg_max = g_ecg._nodes.ecg.max()


mod_k_to_ecg = {
    i % K: top_k_ecg.iloc[i % K, 0]
    for i in range(K)
}
                          

n2_df = cudf.DataFrame({
    'id': [f'{i}_nn' for i in range(N)],
    'ecg': (cudf.Series(cp.random.randint(ecg_min, ecg_max, size=N)) % K).map(mod_k_to_ecg)
})

In [14]:
g2_ecg = g_ecg.nodes(
    cudf.concat([
        g_ecg._nodes.assign(orig=True),
        n2_df.assign(orig=False)
    ])
).edges(cudf.concat([g_ecg._edges, e2_df]))

In [15]:
g2_ecg._nodes.head()

,id,ecg,orig
0,3952_n,125,True
1,3953_n,169,True
2,3954_n,125,True
3,3955_n,12,True
4,3958_n,80,True


In [16]:
g2_ecg._nodes[ g2_ecg._nodes.id.str.contains('nn') ]

,id,ecg,orig
0,0_nn,86,False
1,1_nn,66,False
2,2_nn,19,False
3,3_nn,66,False
4,4_nn,46,False
...,...,...,...
99995,99995_nn,169,False
99996,99996_nn,169,False
99997,99997_nn,66,False
99998,99998_nn,19,False


In [17]:
#g_ecg.plot()

## FA2

In [18]:
g2_fa = g2_ecg.fa2_layout()

num_nodes: 100000
delta_r: 50.0, spacing_s: 5.0
node_start_radii (%s):
%s <class 'cudf.core.series.Series'> 0        13499.564453
1        13499.564453
2        13499.564453
3        13499.564453
4        13499.564453
             ...     
99995    13499.564453
99996    13499.564453
99997    13499.564453
99998    13499.564453
99999    13499.564453
Length: 100000, dtype: float32
node_indices (%s):
%s <class 'cupy.ndarray'> [    0     1     2 ... 99997 99998 99999]
numerator: 0        0
1        0
2        0
3        0
4        0
        ..
99995    0
99996    0
99997    0
99998    0
99999    0
Length: 100000, dtype: int64
Any negative values in numerator? 0
node_start_radii: 0        13499.564453
1        13499.564453
2        13499.564453
3        13499.564453
4        13499.564453
             ...     
99995    13499.564453
99996    13499.564453
99997    13499.564453
99998    13499.564453
99999    13499.564453
Length: 100000, dtype: float32
node_indices: [    0     1     2 ... 99997 9

In [19]:
g2_fa.plot()

## GIB

In [20]:
g2_gib = g2_ecg.group_in_a_box_layout(partition_key='ecg')

bulk_mode layout
size - keys 188
size - center_x_per_partition 188
size - center_y_per_partition 188
size - width_per_partition 188
size - height_per_partition 188
num_nodes: 100000
delta_r: 50.0, spacing_s: 5.0
node_start_radii (%s):
%s <class 'cudf.core.series.Series'> 0        374.814704
1        374.814704
2        374.814704
3        374.814704
4        374.814704
            ...    
99995    373.654712
99996    373.654712
99997    373.654712
99998    373.654712
99999    373.654712
Length: 100000, dtype: float64
node_indices (%s):
%s <class 'cupy.ndarray'> [    0     1     2 ... 99997 99998 99999]
numerator: 0        0
1        0
2        0
3        0
4        0
        ..
99995    0
99996    0
99997    0
99998    0
99999    0
Length: 100000, dtype: int64
Any negative values in numerator? 0
node_start_radii: 0        374.814704
1        374.814704
2        374.814704
3        374.814704
4        374.814704
            ...    
99995    373.654712
99996    373.654712
99997    373.65

In [21]:
g2_gib.plot()

In [22]:
g2_gib._nodes.head(10)

,id,ecg,orig,x,y,degree_in,degree_out,degree,node_idx_relative,nodes_in_ring,cx,cy,node_radii,node_start_radii,ring_numbers,node_widths,node_heights,is_ok_community,expt,type
0,2031_n,19,True,2364.380371,6007.918945,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,force_atlas2
1,2032_n,19,True,2075.984375,6008.221191,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,force_atlas2
2,2033_n,48,True,2632.051758,7668.122559,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,force_atlas2
3,2034_n,124,True,2757.224854,6405.656738,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,force_atlas2
4,2035_n,124,True,2803.757568,6217.627930,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,force_atlas2
5,2036_n,19,True,2454.189941,6218.378418,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,force_atlas2
6,2037_n,48,True,2569.577881,7487.009277,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,force_atlas2
7,2038_n,19,True,2207.141602,6000.337402,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,force_atlas2
8,2039_n,19,True,1993.291016,6101.513672,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,force_atlas2
9,2040_n,48,True,2447.797119,7438.072754,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,force_atlas2


In [23]:
g2_gib._nodes[ ~g2_gib._nodes.orig & g2_gib._nodes.cx.isna() ]

,id,ecg,orig,x,y,degree_in,degree_out,degree,node_idx_relative,nodes_in_ring,cx,cy,node_radii,node_start_radii,ring_numbers,node_widths,node_heights,is_ok_community,expt,type


In [24]:
b = g2_gib._nodes[ ~g2_gib._nodes.orig ]

mask = b.groupby('ecg').cumcount() < 3

b[mask]

,id,ecg,orig,x,y,degree_in,degree_out,degree,node_idx_relative,nodes_in_ring,cx,cy,node_radii,node_start_radii,ring_numbers,node_widths,node_heights,is_ok_community,expt,type
4039,2_nn,19,False,5824.080329,-8107.169922,0,0,0,0,9451,5449.265625,-8107.169922,374.814704,374.814704,0,504.826703,504.826703,False,0.000000,force_atlas2
4040,7_nn,19,False,5824.080246,-8106.920739,0,0,0,1,9451,5449.265625,-8107.169922,374.814704,374.814704,0,504.826703,504.826703,False,0.000000,force_atlas2
4041,12_nn,19,False,5824.079998,-8106.671556,0,0,0,2,9451,5449.265625,-8107.169922,374.814704,374.814704,0,504.826703,504.826703,False,0.000000,force_atlas2
14233,4_nn,46,False,-1927.826336,-12443.355469,0,0,0,0,9576,-2302.089111,-12443.355469,374.262775,374.262775,0,504.083326,504.083326,False,0.000104,force_atlas2
14234,6_nn,46,False,-1927.826417,-12443.109900,0,0,0,1,9576,-2302.089111,-12443.355469,374.262775,374.262775,0,504.083326,504.083326,False,0.000104,force_atlas2
14235,13_nn,46,False,-1927.826658,-12442.864332,0,0,0,2,9576,-2302.089111,-12443.355469,374.262775,374.262775,0,504.083326,504.083326,False,0.000104,force_atlas2
24397,22_nn,48,False,31.573942,-3202.340332,0,0,0,0,10268,-344.598724,-3202.340332,376.172666,376.172666,0,506.655702,506.655702,False,0.000195,force_atlas2
24398,25_nn,48,False,31.573872,-3202.110145,0,0,0,1,10268,-344.598724,-3202.340332,376.172666,376.172666,0,506.655702,506.655702,False,0.000195,force_atlas2
24399,26_nn,48,False,31.573660,-3201.879958,0,0,0,2,10268,-344.598724,-3202.340332,376.172666,376.172666,0,506.655702,506.655702,False,0.000195,force_atlas2
34665,10_nn,62,False,557.042976,-2318.512207,0,0,0,0,10164,-1122.775146,-2318.512207,1679.818122,1679.818122,0,2649.607422,1793.701050,True,0.000295,force_atlas2


In [25]:
h = g2_gib._nodes[ ~g2_gib._nodes.orig ].head(100)

h.node_idx_relative / h.nodes_in_ring

4039    0.000000
4040    0.000106
4041    0.000212
4042    0.000317
4043    0.000423
          ...   
4134    0.010052
4135    0.010158
4136    0.010263
4137    0.010369
4138    0.010475
Length: 100, dtype: float64

In [26]:
#g_gib._complex_encodings

In [27]:
g2_gib._nodes[ ~g2_gib._nodes.orig ].dtypes

id                    object
ecg                    int32
orig                    bool
x                    float64
y                    float64
degree_in              int32
degree_out             int32
degree                 int32
node_idx_relative      int64
nodes_in_ring          int64
cx                   float32
cy                   float32
node_radii           float64
node_start_radii     float64
ring_numbers           int64
node_widths          float64
node_heights         float64
is_ok_community         bool
expt                 float64
type                  object
dtype: object

In [28]:
4 / 9836

0.00040666937779585197

In [29]:
import pandas as pd
pd.Series([4]) / pd.Series([9836])

0    0.000407
dtype: float64

In [30]:
cudf.Series(pd.Series([4]) / pd.Series([9836]))

0    0.000407
dtype: float64

In [31]:
cudf.Series([4]) / cudf.Series([9836])

0    0.000407
dtype: float64